In [6]:
from SAM import SAM
import numpy as np
import pandas as pd
import sklearn.metrics as met
import utilities as ut
import utilities_full as ut2
from sklearn.preprocessing import Normalizer
import pickle

def DARMANIS(**kwargs):
    sam = SAM()
    sam.load_data('darmanis/darmanis_data.csv', **kwargs)
    sam.load_obs_annotations('darmanis/darmanis_ann.csv')
    sam.preprocess_data()
    return sam

def WANG(**kwargs):
    sam = SAM()
    sam.load_data('final_datasets/GSE83139/wang_data_sparse.p', **kwargs)
    sam.preprocess_data(filter_genes=False,min_expression=1)
    A = pd.read_csv('final_datasets/GSE83139/wang_ann.csv',header=None,index_col=0)    
    A.index = A.index.astype("<U100")
    sam.adata.obs['ann'] = A
    sam.adata.var_names_make_unique()
    return sam

def human1(**kwargs):
    sam = SAM()
    sam.load_data('final_datasets/GSE84133_1/human1_sparse.p')
    sam.preprocess_data()
    sam.load_obs_annotations('final_datasets/GSE84133_1/human1_ann.csv')
    return sam


def human2(**kwargs):
    sam = SAM()
    sam.load_data('final_datasets/GSE84133_2/human2_sparse.p')
    sam.preprocess_data()
    sam.load_obs_annotations('final_datasets/GSE84133_2/human2_ann.csv')
    return sam

def human3(**kwargs):
    sam = SAM()
    sam.load_data('final_datasets/GSE84133_3/human3_sparse.p')
    sam.preprocess_data()
    sam.load_obs_annotations('final_datasets/GSE84133_3/human3_ann.csv')
    return sam
def human4(**kwargs):
    sam = SAM()
    sam.load_data('final_datasets/GSE84133_4/human4_sparse.p')
    sam.preprocess_data()
    sam.load_obs_annotations('final_datasets/GSE84133_4/human4_ann.csv')
    return sam


def KOH(**kwargs):
    sam = SAM()
    sam.load_data('final_datasets/SRP073808/SRP073808_data.csv',
                            **kwargs)
    sam.load_obs_annotations('final_datasets/SRP073808/SRP073808_ann.csv')
    sam.preprocess_data()
    return sam


def SEGER(**kwargs):
    sam=SAM()
    sam.load_data('final_datasets/seger/seger_sparse.p')
    sam.load_obs_annotations('final_datasets/seger/seger_ann.csv')
    sam.preprocess_data()
    return sam


def MURARO(**kwargs):
    sam=SAM()
    sam.load_data('final_datasets/muraro/muraro_sparse.p')
    sam.load_obs_annotations('final_datasets/muraro/muraro_ann.csv')
    sam.preprocess_data()
    return sam

In [10]:

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
from rpy2.robjects import r
import matplotlib.pyplot as plt

StrVector = robjects.StrVector
rpy2.robjects.numpy2ri.activate()
SingleCellExperiment = r(''' 
  f <- function(x){
    sce <- SingleCellExperiment(assays = list(counts = x))
  }
  ''')

SC3 = r(''' 
        function(data,genes,k,genefilter){
              sce <- SingleCellExperiment(
                assays = list(
                  counts = data,
                  logcounts = log2(data + 1)
                )
              )
              rowData(sce)$feature_symbol <- genes
              sce <- sce[!duplicated(rowData(sce)$feature_symbol), ]
              if(k==0){
                  sce <- sc3_estimate_k(sce)
                  k <- metadata(sce)$sc3$k_estimation
                  if (k > 50){
                  k = 50
                  }
                 }
              print(k)
              sce <- sc3(sce, ks = k, biology = FALSE,gene_filter=genefilter)
              col_data <- as.matrix(colData(sce))
              nlist <- list("rdata" = col_data, "k" = k)
         }
        ''')


SIMLR = r('''
          function(data,genes,k){
              b<-as.data.frame(data)
              rownames(b)<-genes
              if(k==0){
                  NUMC = 2:10
                  res_example = SIMLR_Estimate_Number_of_Clusters(X = b,NUMC=NUMC,cores.ratio=1)            
                  c1=NUMC[which.min(res_example$K1)]
                  c2=NUMC[which.min(res_example$K2)]
                  c = min(c(c1,c2))
              }else{
                      c=k
              }
              if( c > 50){
              c = 50
              
              }
              print(c)              
              
              res_hsc <- SIMLR(X =  b, normalize = T, c = c)
              res_hsc
        
              nlist <- list("rdata" = res_hsc, "k" = c)
          }

          ''')
SCE = importr('SingleCellExperiment')
scater = importr('scater')
simlr = importr('SIMLR')
sc3 = importr('SC3')

def SEURAT(adata):
    pca,_,_,_ = ut2.do_SEURAT4(adata.copy(),NN=3000)
    cl = hdbknn(Normalizer().fit_transform(pca))
    return pca,cl

def SEURAT2(adata,ng,npc):
    pca,_,_,_ = ut2.do_SEURAT4(adata.copy(),npcs=npc,NN=ng)
    cl = hdbknn(Normalizer().fit_transform(pca))
    return pca,cl

def hdbknn(X):
    import hdbscan
    k=20

    hdb = hdbscan.HDBSCAN(metric='euclidean')

    cl = hdb.fit_predict(Normalizer().fit_transform(X))

    idx0 = np.where(cl != -1)[0]
    idx1 = np.where(cl == -1)[0]
    if idx1.size > 0 and idx0.size > 0:
        xcmap = ut.generate_euclidean_map(X[idx0, :], X[idx1, :])
        knn = np.argsort(xcmap.T, axis=1)[:, :k]
        nnm = np.zeros(xcmap.shape).T
        nnm[np.tile(np.arange(knn.shape[0])[:, None],
                    (1, knn.shape[1])).flatten(),
            knn.flatten()] = 1
        nnmc = np.zeros((nnm.shape[0], cl.max() + 1))
        for i in range(cl.max() + 1):
            nnmc[:, i] = nnm[:, cl[idx0] == i].sum(1)

        cl[idx1] = np.argmax(nnmc, axis=1)

    return cl
def ari(x, y):
    return met.adjusted_rand_score(x, y)
def colabeling(a,b):
    au,acu = np.unique(a,return_counts=True)
    bu,bcu = np.unique(b,return_counts=True)
    co = np.zeros((au.size,bu.size))
    
    for i in range(bu.size):
        x1,x2 = np.unique(a[b==bu[i]],return_counts=True)
        co[np.in1d(au,x1),i] += x2# / bcu[i]
    #coa = co*1/acu[:,None]
    #cob = co*1/bcu[None,:]

    midx = np.argmax(co,axis=0)
    scores=np.zeros(midx.size)
    for i in range(midx.size):
        #scores[i] = (co[midx[i],i]/co[:,i].sum() + co[midx[i],i]/co[midx[i],:].sum())/2    
        scores[i] = co[midx[i],i] / max(bcu[i],acu[midx[i]])        
    return co,scores

In [8]:
preproc = ['Normalizer','Normalizer']+['StandardScaler',]*4+['Normalizer',]*3
funcs = [DARMANIS(),WANG(),human1(),human2(),human3(),human4(),KOH(),SEGER(),MURARO()]
names = ['DARMANIS','WANG','human1','human2','human3','human4','KOH','SEGER','MURARO']

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


Converting sparse matrix to csr format...
Converting sparse matrix to csr format...


In [9]:
ngenes = [500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,-1]
npcs = [6,10,15,20,25,30,35,40,45,50]
RECORDd = pickle.load(open('paper_scripts/seurat_param_sweep_bigger_fixed.p','rb'))
nge=[]
npc=[]
for i in names:
    ind = np.where(RECORDd[i]>= RECORDd[i].max())
    n1,n2 = ngenes[ind[0][0]],npcs[ind[1][0]]
    nge.append(n1)
    npc.append(n2)
    
nge[1]=3000
npc[1]=6

In [ ]:
(AMIsc,AMIsi) = pickle.load(open('paper_scripts/amiari_sc3sim_uncorrupted2.p','rb'))
AMIsc = AMIsc[:,np.arange(10)!=7]
AMIsi = AMIsi[:,np.arange(10)!=7]
ksc = AMIsc[-1,:]
ksi = AMIsi[-1,:]

C1=[]
C2=[]
C3=[]
for I in range(len(names)):
    print(names[I])
    sam = funcs[I]
    
    ge = np.array(list(sam.adata.var_names))
    
    print('SC3')    
    if names[I] == 'WANG':
        cluster_labels1 = list(list(SC3(sam.adata_raw.X.A.T,ge,ksc[I],False))[0])
    else:
        cluster_labels1 = list(list(SC3(sam.adata_raw.X.A.T,ge,ksc[I],True))[0])
        
    print('SIMLR')
    cluster_labels2 = list(list(SIMLR(sam.adata.X.A.T,ge,ksi[I]))[0][0][0])
    print('Seurat')
    _,cluster_labels3 = SEURAT(sam.adata_raw.copy())
    
    C1.append(cluster_labels1)
    C2.append(cluster_labels2)
    C3.append(cluster_labels3)
    
    print('SC3 - ' + str(ari(funcs[I].adata.obs.iloc[:,0],cluster_labels1)))
    print('SIMLR - ' + str(ari(funcs[I].adata.obs.iloc[:,0],cluster_labels2)))
    print('Seurat - ' + str(ari(funcs[I].adata.obs.iloc[:,0],cluster_labels3)))

In [12]:
C4=[]
for i in range(len(funcs)):
    adata = funcs[i].adata_raw.copy()
    if i == 1:
        adata.X[adata.X<1]=0
        adata.X.eliminate_zeros()
        
    _,cl = SEURAT2(adata,nge[i],npc[i])
    C4.append(cl)
    

2000
3000
4000
4500
2500
5000
6000
3500
2000


In [ ]:
C0=[]
for i in range(len(funcs)):
    print(i)
    funcs[i].run(preprocessing=preproc[i])
    funcs[i].hdbknn_clustering(npcs=15)
    C0.append(funcs[i].adata.obs['hdbscan_clusters'].get_values())
    

In [16]:
(C0,C1,C2,C3,C4) = pickle.load(open('paper_scripts/cluster_assignments_sc3_sim_seur_opt.p','rb'))

In [28]:
import scipy as sp

S = []
for I in range(9):
    sam = funcs[I]
    cl0 = C0[I]
    cl1 = C1[I]
    cl2 = C2[I]
    cl3 = C3[I]
    cl4 = C4[I]

    CS = [cl0,cl1,cl2,cl3, cl4]
    ann = sam.adata.obs.iloc[:,0].get_values()
    annu,annuc = np.unique(sam.adata.obs.iloc[:,0],return_counts=True)

    x=[]
    scores = np.zeros((len(CS),annu.size))
    
    
    co = colabeling(ann,ann)[0]
    NUMERATOR2 = sp.special.binom(co,2).sum(0) - (sp.special.binom(co.sum(1),2).sum()*sp.special.binom(co.sum(0),2)) / sp.special.binom(cl0.size,2)
    DENOMINATOR2 = (sp.special.binom(co.sum(1),2).sum()+sp.special.binom(co.sum(0),2).sum()) * 0.5 - (sp.special.binom(co.sum(1),2).sum()*sp.special.binom(co.sum(0),2).sum())/ sp.special.binom(cl0.size,2)        
    gt = NUMERATOR2/DENOMINATOR2
    
    for m in range(len(CS)):
        c = np.array(CS[m])
        cu = np.unique(c)
        co = colabeling(c,ann)[0]
        NUMERATOR = sp.special.binom(co,2).sum(0) - (sp.special.binom(co.sum(1),2).sum()*sp.special.binom(co.sum(0),2)) / sp.special.binom(cl0.size,2)
        DENOMINATOR = (sp.special.binom(co.sum(1),2).sum()+sp.special.binom(co.sum(0),2).sum()) * 0.5 - (sp.special.binom(co.sum(1),2).sum()*sp.special.binom(co.sum(0),2).sum())/ sp.special.binom(cl0.size,2)
        scores[m,:] = (NUMERATOR / DENOMINATOR)# / (NUMERATOR2 / DENOMINATOR2)# * c.size / np.unique(ann,return_counts=True)[1]
        #print((NUMERATOR2 / DENOMINATOR2).sum())
    S.append(pd.DataFrame(data = np.hstack((np.append(annuc,annuc.sum())[:,None],np.vstack((scores.T,scores.T.sum(0))),np.append(gt,gt.sum())[:,None])), columns = ['# cells','SAM','SC3','SIMLR','Seurat', 'Seurat optimized', ' Ground truth'], index = np.append(annu,'ARI (Total)')))
                

In [21]:
names = ['Darmanis','Wang','Baron1','Baron2','Baron3','Baron4','Koh','Segerstolpe','Muraro']
with pd.ExcelWriter("/media/storage/dbox/Dropbox/paper_scripts/ARI_TABLES2.xlsx") as writer:  
    for i in range(len(S)):
        S[i].index.name = names[i] + ' labels'
        S[i].to_excel(writer,sheet_name=names[i])

In [39]:
sam=SAM()
sam.load_data('schisto2.5_tpm.csv')
sam.preprocess_data()
sam.run()
sam.louvain_clustering()

pca,_ = SEURAT(sam.adata_raw.copy())
import scipy as sp
cl = sam.leiden_clustering(X=sp.sparse.csr_matrix(ut.dist_to_nn(ut.compute_distances(pca,'correlation'),20)))



RUNNING SAM
Iteration: 0, Convergence: 0.4526962542729544
Iteration: 1, Convergence: 0.12612906385641998
Iteration: 2, Convergence: 0.07305737834954922
Iteration: 3, Convergence: 0.021107561918671688
Iteration: 4, Convergence: 0.008076236324138297
Computing the UMAP embedding...
Elapsed time: 3.384687662124634 seconds
3000


In [40]:
ari(cl,sam.adata.obs['louvain_clusters'])

0.32706247385306214